<a href="https://colab.research.google.com/github/EoghanPower/pr/blob/main/Create_a_Tweets_DataFramefinished.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:

#Creating a Data Frame of Tweets on S&P 500 Companies

In [32]:
# import libraries
import tweepy, json 
from textblob import TextBlob 
from wordcloud import wordcloud
import pandas as pd
import numpy as np 
import re
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import webbrowser
import time 
import requests
import requests_oauthlib
import os

In [33]:
#authentication store 0auth credentials in variables
consumer_key = 'eHrpqTZVq5bfGdjC8juN7fSCs'
consumer_secret = 'itfojeERaMFXtwQFI1emQNT3vwCHs414OJdOXlpN2RaBAw3MIz'
AccessToken = '264845011-WhC9gKJlC3TayNsXg4ViBIuwaK0D5Sof3lxPTZeN'
AccessTokenSecret = 'dE5difJJmYP5UYllqeQwTsovcoXyvco9Ww2QMxiR365D3'

In [34]:
#Pass 0auth details to tweepy's 0auth handlet 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(AccessToken, AccessTokenSecret)

In [35]:
##SetAPIobject
api = tweepy.API(auth, wait_on_rate_limit= True)

In [36]:
#StreamListener
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets.txt", "w")

    def on_status(self, status):
        tweet = status._json
        self.file.write( json.dumps(tweet) + '\n' )
        self.num_tweets += 1
        if self.num_tweets < 10001:
            return True
        else:
            return False
        self.file.close()

    def on_error(self, status):
        print(status)

In [37]:
#initialize stream listener
l = MyStreamListener()
#Create stream object with authentication
stream = tweepy.Stream(auth, l)
#Filter Twitter Streams to capture data by the keywords:
stream.filter(languages = ['en'],track=['Apple', 'Google', 'Microsoft','Amazon',  'Facebook', 'JPMorgan',   'Johnson & Johnson', '  Exxon', 'Bank of America', 'Wal-Mart, Wells Fargo', 'Berkshire Hathaway', 'AT&T ', 'Tesla', 'Gamestop' ])

In [38]:
#import package
import json

#String of path to file: tweets_data_path
tweets_data_path = 'tweets.txt'

#initialize empty list to store tweets: tweets_data
tweets_data = []

In [39]:
#open connection to file
tweets_file = open(tweets_data_path, 'r')

In [40]:
# Read in tweets and store in list: tweets_data
for line in tweets_file:
  tweet = json.loads(line)
  tweets_data.append(tweet)

In [41]:
#close connection to file
tweets_file.close()


In [42]:
print(tweets_data[0].keys())

dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])


In [43]:
# Build DataFrame of tweet texts and languages
df = pd.DataFrame(tweets_data, columns= ['text'])

In [44]:
print(df.head(10))

                                                text
0  Self awareness? Alright, Captain Tom, you’ve g...
1  RT @beatsbylonza: El HJEFE Podcast Episode 2. ...
2  ICYMI. Looks like it was a nice day for our to...
3  Livestream links for tonight's girls and boys ...
4  RT @Rigid3d: Some more renders. \n\nhttps://t....
5  @AdAltum @thetimes Amazon is incorporated in m...
6  RT @NatapanuN: Thailand Pass will be closed fo...
7  @author_ankita @jgmacleodauthor https://t.co/F...
8  @TaylorOgan The bus driving ahead of Tesla in ...
9  RT @CBSNews: This Amazon delivery driver went ...


In [45]:
#clean the tweets
def cleanTwt(twt): 
  twt = re.sub('#apple','Apple', twt)
  twt = re.sub('#google','Google', twt) 
  twt = re.sub('#microsoft','Microsoft', twt) 
  twt = re.sub('#amazon','Amazon', twt) 
  twt = re.sub('#facebook','Facebook', twt) 
  twt = re.sub('#JPMorgan','JPMorgan', twt) 
  twt = re.sub('#Johnson&Johnson','Johnson & Johnson', twt) 
  twt = re.sub('#Exxon','Exxon', twt) 
  twt = re.sub('#BankofAmerica','Bank of America', twt) 
  twt = re.sub('#Wal-Mart','Wal-Mart', twt) 
  twt = re.sub('#Wells Fargo','Wells Fargo', twt) 
  twt = re.sub('#BerkshireHathaway','Berkshire Hathaway', twt) 
  twt = re.sub('#Gamestop','Gamestop', twt) 
  twt = re.sub('#Tesla ','Tesla', twt)#removes the # 
  twt =re.sub('#[A-Za-z0-9]+', '',twt) #removes any strings with a # 
  twt = re.sub('\\n', '', twt) #removes the '\n' string
  twt = re.sub('https?:\/\/\S+', '', twt) #removes any hyperlinks
  twt = re.sub('RT', '', twt) #cleans retweets
  twt = re.sub('@[A-Za-z0-9]+', ',', twt)#Removes tagged accounts
  return twt

In [46]:
#clean them using the above function

In [47]:
df['Cleaned_Tweets'] = df['text'].apply(cleanTwt)

In [48]:
#show the set 
df.head(10)

,text,Cleaned_Tweets
0,"Self awareness? Alright, Captain Tom, you’ve g...","Self awareness? Alright, Captain Tom, you’ve g..."
1,RT @beatsbylonza: El HJEFE Podcast Episode 2. ...,",: El HJEFE Podcast Episode 2. Spotify, App..."
2,ICYMI. Looks like it was a nice day for our to...,ICYMI. Looks like it was a nice day for our to...
3,Livestream links for tonight's girls and boys ...,Livestream links for tonight's girls and boys ...
4,RT @Rigid3d: Some more renders. \n\nhttps://t....,",: Some more renders."
5,@AdAltum @thetimes Amazon is incorporated in m...,", , Amazon is incorporated in many countries, ..."
6,RT @NatapanuN: Thailand Pass will be closed fo...,",: Thailand Pass will be closed for all new T..."
7,@author_ankita @jgmacleodauthor https://t.co/F...,",_ankita , you so much. Currently working on ..."
8,@TaylorOgan The bus driving ahead of Tesla in ...,", The bus driving ahead of Tesla in the right ..."
9,RT @CBSNews: This Amazon delivery driver went ...,",: This Amazon delivery driver went above and..."


In [49]:
#create a function to get subjectivity
def getSubjectivity(twt):
  return TextBlob(twt).sentiment.subjectivity
#create a function to get polarity
def getPolarity(twt):
  return TextBlob(twt).sentiment.polarity


In [50]:
#create two new columns called subjectivity and polarity
df['Subjectivity'] = df['Cleaned_Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Cleaned_Tweets'].apply(getPolarity)

In [51]:
#show data
df.head(20)

,text,Cleaned_Tweets,Subjectivity,Polarity
0,"Self awareness? Alright, Captain Tom, you’ve g...","Self awareness? Alright, Captain Tom, you’ve g...",0.000000,0.000000
1,RT @beatsbylonza: El HJEFE Podcast Episode 2. ...,",: El HJEFE Podcast Episode 2. Spotify, App...",0.000000,0.000000
2,ICYMI. Looks like it was a nice day for our to...,ICYMI. Looks like it was a nice day for our to...,0.375000,0.275000
3,Livestream links for tonight's girls and boys ...,Livestream links for tonight's girls and boys ...,0.000000,0.000000
4,RT @Rigid3d: Some more renders. \n\nhttps://t....,",: Some more renders.",0.500000,0.500000
5,@AdAltum @thetimes Amazon is incorporated in m...,", , Amazon is incorporated in many countries, ...",0.500000,0.500000
6,RT @NatapanuN: Thailand Pass will be closed fo...,",: Thailand Pass will be closed for all new T...",0.218182,0.012121
7,@author_ankita @jgmacleodauthor https://t.co/F...,",_ankita , you so much. Currently working on ...",0.400000,0.000000
8,@TaylorOgan The bus driving ahead of Tesla in ...,", The bus driving ahead of Tesla in the right ...",0.535714,0.285714
9,RT @CBSNews: This Amazon delivery driver went ...,",: This Amazon delivery driver went above and...",0.050000,0.000000


In [52]:
#Create a function to get the sentiment text
def getSentiment(score):
  if score <0:
    return 'Negative'
  elif score == 0:
      return 'Neutral'
  else:
      return 'Positive'

In [53]:
#Create column to store the text sentiment
df['Sentiment'] = df['Polarity'].apply(getSentiment)
#Show the data
df.head(7)

,text,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,"Self awareness? Alright, Captain Tom, you’ve g...","Self awareness? Alright, Captain Tom, you’ve g...",0.000000,0.000000,Neutral
1,RT @beatsbylonza: El HJEFE Podcast Episode 2. ...,",: El HJEFE Podcast Episode 2. Spotify, App...",0.000000,0.000000,Neutral
2,ICYMI. Looks like it was a nice day for our to...,ICYMI. Looks like it was a nice day for our to...,0.375000,0.275000,Positive
3,Livestream links for tonight's girls and boys ...,Livestream links for tonight's girls and boys ...,0.000000,0.000000,Neutral
4,RT @Rigid3d: Some more renders. \n\nhttps://t....,",: Some more renders.",0.500000,0.500000,Positive
5,@AdAltum @thetimes Amazon is incorporated in m...,", , Amazon is incorporated in many countries, ...",0.500000,0.500000,Positive
6,RT @NatapanuN: Thailand Pass will be closed fo...,",: Thailand Pass will be closed for all new T...",0.218182,0.012121,Positive


In [54]:
df.drop(columns = 'text')

,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,"Self awareness? Alright, Captain Tom, you’ve g...",0.000,0.000,Neutral
1,",: El HJEFE Podcast Episode 2. Spotify, App...",0.000,0.000,Neutral
2,ICYMI. Looks like it was a nice day for our to...,0.375,0.275,Positive
3,Livestream links for tonight's girls and boys ...,0.000,0.000,Neutral
4,",: Some more renders.",0.500,0.500,Positive
...,...,...,...,...
9996,",: Great gift for 24 hours from now ✅😉📌 Buy $...",0.750,0.800,Positive
9997,The November market report is here!Send me a D...,0.000,0.000,Neutral
9998,",: Big Tech has to be stopped to protect our ...",0.100,0.000,Neutral
9999,Facebook …,0.000,0.000,Neutral


In [55]:
#drop columns to make it easier to read
df.head()

,text,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,"Self awareness? Alright, Captain Tom, you’ve g...","Self awareness? Alright, Captain Tom, you’ve g...",0.000,0.000,Neutral
1,RT @beatsbylonza: El HJEFE Podcast Episode 2. ...,",: El HJEFE Podcast Episode 2. Spotify, App...",0.000,0.000,Neutral
2,ICYMI. Looks like it was a nice day for our to...,ICYMI. Looks like it was a nice day for our to...,0.375,0.275,Positive
3,Livestream links for tonight's girls and boys ...,Livestream links for tonight's girls and boys ...,0.000,0.000,Neutral
4,RT @Rigid3d: Some more renders. \n\nhttps://t....,",: Some more renders.",0.500,0.500,Positive


In [56]:
df.tail()

,text,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
9996,RT @FaceMetaaverse: Great gift for 24 hours fr...,",: Great gift for 24 hours from now ✅😉📌 Buy $...",0.75,0.8,Positive
9997,The November market report is here!\n\nSend me...,The November market report is here!Send me a D...,0.00,0.0,Neutral
9998,RT @RepKenBuck: Big Tech has to be stopped to ...,",: Big Tech has to be stopped to protect our ...",0.10,0.0,Neutral
9999,#linkedin #twitter #facebook #gifting #instagr...,Facebook …,0.00,0.0,Neutral
10000,GE Drip Coffee Maker With Timer | 12-Cup Glass...,GE Drip Coffee Maker With Timer | 12-Cup Glass...,0.00,0.0,Neutral


In [57]:
#export the dataframe to sheets
#export to google sheets part one authentication
!pip install gspread
from google.colab import auth as au
au.authenticate_user()

In [75]:
!pip install df2gspread

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [59]:
import gspread
from oauth2client.client import GoogleCredentials

In [60]:
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [66]:
from gspread.models import Worksheet
#export to google sheets part two export
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [78]:
from df2gspread import df2gspread as d2g

In [83]:
spreadsheet_key = '1uO1co8iCP6JZDg84WllMROpvI2O0yLMFfBeK7jMMisQ'
wks_name = 'Master'
d2g.upload(df, spreadsheet_key, wks_name, credentials= GoogleCredentials.get_application_default(), row_names=True)

<Worksheet 'Master' id:588663387>